# Obtaining New York Neighborhoods

In [1]:
#importing the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import re #regular expessions

! pip install beautifulsoup4
! pip install requests
import requests #request online data
from bs4 import BeautifulSoup as bs # library for navigating html files

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [18]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [19]:
VERSION = '20180604'
LIMIT = 30

In [3]:
# obtained newyork_data.json from a previous lab
with open('newyork_data.json') as newyork_json:
    newyork_data = json.load(newyork_json)

In [4]:
neighborhoods = newyork_data['features']

In [5]:
ny_hoods = pd.DataFrame(columns = ['Borough','Neighborhood','Hood_Lat','Hood_Lng'])
ny_hoods

,Borough,Neighborhood,Hood_Lat,Hood_Lng


In [6]:
for data in neighborhoods:
    try:
        borough = data['properties']['borough']
    except:
        borough = 'No Data'
    try:
        hood = data['properties']['name']
    except:
        hood = 'No Data'
    try:
        geom = data['geometry']['coordinates']
        lat = geom[1]
        lng = geom[0]
    except:
        lat = 'No Data'
        lng = 'No Data'
    ny_hoods = ny_hoods.append({'Borough': borough,'Neighborhood':hood,'Hood_Lat':lat,'Hood_Lng':lng}, ignore_index=True)
ny_hoods.head()

,Borough,Neighborhood,Hood_Lat,Hood_Lng
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
borough_longitude_list = list()
borough_latitude_list = list()
for borough in ny_hoods['Borough']:
    try:
        address = '{}, New York City, NY'.format(borough)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        borough_longitude_list.append(longitude)
        borough_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        borough_longitude_list.append(longitude)
        borough_latitude_list.append(latitude)
print('areas printed')

areas printed


In [8]:
ny_hoods.insert(loc = 1, column = 'Borough_Lat', value = borough_latitude_list)
ny_hoods.insert(loc = 2, column = 'Borough_Lng', value = borough_longitude_list)
ny_hoods.head()

,Borough,Borough_Lat,Borough_Lng,Neighborhood,Hood_Lat,Hood_Lng
0,Bronx,40.846651,-73.878594,Wakefield,40.894705,-73.847201
1,Bronx,40.846651,-73.878594,Co-op City,40.874294,-73.829939
2,Bronx,40.846651,-73.878594,Eastchester,40.887556,-73.827806
3,Bronx,40.846651,-73.878594,Fieldston,40.895437,-73.905643
4,Bronx,40.846651,-73.878594,Riverdale,40.890834,-73.912585


In [9]:
#to prove df is storing correct values and not rounding them
for borlat in ny_hoods['Borough_Lat'][[0,5,30,50]]:
    print(borlat)

40.8466508
40.8466508
40.8466508
40.6501038


In [10]:
for index, thing in enumerate(ny_hoods['Borough']):
    if index == 50:
        address = '{}, New York City, NY'.format(thing)
        print(thing)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print(latitude)

Brooklyn
40.6501038


In [11]:
ny_hoods = ny_hoods.set_index('Borough')

In [12]:
ny_hoods.to_csv('newyork_geodata.csv')

In [17]:
#folium.map cannot have string values for latitude and longitude
for lat,lng in zip(ny_hoods['Hood_Lat'], ny_hoods['Hood_Lng']):
    if type(lat)== float or type(lat)==int:
        latitude=lat
        longitude=lng
        
#map each neighborhood
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, neighborhood in zip(ny_hoods['Hood_Lat'], ny_hoods['Hood_Lng'], ny_hoods['Neighborhood']):
    try:
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_newyork)
    except:
        continue
map_newyork